##### Copyright 2024 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine-tune Gemma models in Keras using LoRA

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/lora_tuning"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/335"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />Open in Vertex AI</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

Gemma is a family of lightweight, state-of-the art open models built from the same research and technology used to create the Gemini models.

Large Language Models (LLMs) like Gemma have been shown to be effective at a variety of NLP tasks. An LLM is first pre-trained on a large corpus of text in a self-supervised fashion. Pre-training helps LLMs learn general-purpose knowledge, such as statistical relationships between words. An LLM can then be fine-tuned with domain-specific data to perform downstream tasks (such as sentiment analysis).

LLMs are extremely large in size (parameters in the order of millions). Full fine-tuning (which updates all the parameters in the model) is not required for most applications because typical fine-tuning datasets are relatively much smaller than the pre-training datasets.

[Low Rank Adaptation (LoRA)](https://arxiv.org/abs/2106.09685){:.external} is a fine-tuning technique which greatly reduces the number of trainable parameters for downstream tasks by freezing the weights of the model and inserting a smaller number of new weights into the model. This makes training with LoRA much faster and more memory-efficient, and produces smaller model weights (a few hundred MBs), all while maintaining the quality of the model outputs.

This tutorial walks you through using KerasNLP to perform LoRA fine-tuning on a Gemma 2B model using the [Databricks Dolly 15k dataset](https://www.kaggle.com/datasets/databricks/databricks-dolly-15k){:.external}. This dataset contains 15,000 high-quality human-generated prompt / response pairs specifically designed for fine-tuning LLMs.

## Setup

### Get access to Gemma

To complete this tutorial, you will first need to complete the setup instructions at [Gemma setup](https://ai.google.dev/gemma/docs/setup). The Gemma setup instructions show you how to do the following:

Gemma models are hosted by Kaggle. To use Gemma, request access on Kaggle:

- Sign in or register at [kaggle.com](https://www.kaggle.com)
- Open the [Gemma model card](https://www.kaggle.com/models/google/gemma) and select _"Request Access"_
- Complete the consent form and accept the terms and conditions


### Install dependencies

Install Keras, KerasNLP, and other dependencies.

In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


### Select a backend

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. Using Keras 3, you can run workflows on one of three backends: TensorFlow, JAX, or PyTorch.

For this tutorial, configure the backend for JAX.

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [3]:
import keras
import keras_nlp

2024-04-20 06:30:07.418215: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 06:30:07.418315: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 06:30:07.538996: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load Dataset

Preprocess the data. This tutorial uses a subset of 1000 training examples to execute the notebook faster. Consider using more training data for higher quality fine-tuning.

In [5]:
import json
data = []
with open('/kaggle/input/databricks-dolly-15k/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

## Load Model

KerasNLP provides implementations of many popular [model architectures](https://keras.io/api/keras_nlp/models/){:.external}. In this tutorial, you'll create a model using `GemmaCausalLM`, an end-to-end Gemma model for causal language modeling. A causal language model predicts the next token based on previous tokens.

Create the model using the `from_preset` method:

In [36]:
import pandas as pd
df=pd.read_csv('/kaggle/input/newtrain/train_dataset.csv')
df=df.sample(frac=1).reset_index(drop=True)
df

,query,target,type
0,Some virtual machines are experiencing high CP...,"{'Skeleton': 'High CPU Usage', 'Arrow1': 'Hard...",fishbone
1,Citrix Virtual Apps sessions are timing out fr...,"{'Cause1': 'Server overload', 'Cause2': 'Insuf...",causecode
2,The load balancer is not distributing traffic ...,{'why1question': 'Why is the load balancer not...,5_why
3,OEM is not able to monitor custom metrics for ...,{'businessImpact': 'Improved application perfo...,sip
4,Users are reporting issues with connecting to ...,"{'Cause1': 'Printer driver issue', 'Cause2': '...",causecode
...,...,...,...
363,Several users have reported missing or incompl...,"{'Cause1': 'Data input errors', 'Cause2': 'Sys...",causecode
364,How can I configure NAT on my Juniper router?,"{'Cause1': 'Incorrect configuration', 'Cause2'...",causecode
365,The deployment process has encountered errors ...,{'businessImpact': 'Improved deployment proces...,sip
366,Several scheduled Tableau data extracts have f...,{'businessImpact': 'Improved accuracy and reli...,sip


In [41]:
def addprompt(row):
    if row['type']=='causecode':
        prompt=f'''Instruction: 
you are an ITSM administrator expert in json response,you have to provide nine casues for following user query :"{row['query']}",response must be only json object.'''+f'''Response:
{row['target']}'''
    elif row['type']=='fishbone':
        prompt=f'''Instruction: 
you are an ITSM administrator expert in json response,you have to provide fishbone analysis for following user query :"{row['query']}",response must be only json object.'''+f'''Response:
{row['target']}'''
    elif row['type']=='sip':
        prompt=f'''Instruction: 
you are an ITSM administrator expert in json response,you have to provide service improvment proposal for following user query :"{row['query']}",response must be only json object.'''+f'''Response:
{row['target']}'''
    else:
        prompt=f'''Instruction: 
you are an ITSM administrator expert in json response,you have to provide five why analysis for following user query :"{row['query']}" ,response must be only json object.'''+f'''Response:
{row['target']}'''
            
    return prompt

In [42]:
df['feed']=df.apply(lambda x:addprompt(x),axis=1)
train_data=df['feed'].tolist()

In [40]:
# train_data

In [60]:
# res_data=data
# res_data

In [43]:
data=train_data

In [75]:
pip show keras_nlp

Name: keras-nlp
Version: 0.9.3
Summary: Industry-strength Natural Language Processing extensions for Keras.
Home-page: https://github.com/keras-team/keras-nlp
Author: Keras team
Author-email: keras-nlp@google.com
License: Apache License 2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: absl-py, dm-tree, kagglehub, keras-core, numpy, packaging, regex, rich, tensorflow-text
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

The `from_preset` method instantiates the model from a preset architecture and weights. In the code above, the string "gemma_2b_en" specifies the preset architecture — a Gemma model with 2 billion parameters.

NOTE: A Gemma model with 7
billion parameters is also available. To run the larger model in Colab, you need access to the premium GPUs available in paid plans. Alternatively, you can perform [distributed tuning on a Gemma 7B model](https://ai.google.dev/gemma/docs/distributed_tuning) on Kaggle or Google Cloud.

## Inference before fine tuning

In this section, you will query the model with various prompts to see how it responds.


### Europe Trip Prompt

Query the model for suggestions on what to do on a trip to Europe.

In [46]:
r="Instruction: you are an ITSM administrator expert in json response,you have to provide nine casues for following user query :Users are reporting intermittent connectivity issues when accessing applications on the AKS cluster you have to fill following json body {'Cause1':... ,'Cause2':... ,'Cause3':... ,'Cause4':... ,'Cause5':... ,'Cause6':... ,'Cause7':... ,'Cause8':... ,'Cause9':...} respond only json object'''+f'''Response:"

In [47]:
prompt = template.format(
    instruction=r,
    response="",
)
print(gemma_lm.generate(prompt, max_length=512))

Instruction:
Instruction: you are an ITSM administrator expert in json response,you have to provide nine casues for following user query :Users are reporting intermittent connectivity issues when accessing applications on the AKS cluster you have to fill following json body {'Cause1':... ,'Cause2':... ,'Cause3':... ,'Cause4':... ,'Cause5':... ,'Cause6':... ,'Cause7':... ,'Cause8':... ,'Cause9':...} respond only json object'''+f'''Response:

Response:
{'Cause1': 'Network Latency', 'Cause2': 'Application Compatibility', 'Cause3': 'Resource Availability', 'Cause4': 'Security Restrictions', 'Cause5': 'Configuration Issues', 'Cause6': 'Hardware Failures', 'Cause7': 'Insufficient Bandwidth', 'Cause8': 'Application Load Balancing', 'Cause9': 'User Error', 'Solution': 'Review network latency and application compatibility to ensure optimal performance. Ensure resource availability by monitoring and optimizing AKS cluster resources. Address security restrictions by reviewing and updating securit

The model just responds with a recommendation to take a trip to Europe.

### ELI5 Photosynthesis Prompt

Prompt the model to explain photosynthesis in terms simple enough for a 5 year old child to understand.


In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

The responses contains words that might not be easy to understand for a child such as chlorophyll, glucose, etc.

## LoRA Fine-tuning

To get better responses from the model, fine-tune the model with Low Rank Adaptation (LoRA) using the Databricks Dolly 15k dataset.

The LoRA rank determines the dimensionality of the trainable matrices that are added to the original weights of the LLM. It controls the expressiveness and precision of the fine-tuning adjustments.

A higher rank means more detailed changes are possible, but also means more trainable parameters. A lower rank means less computational overhead, but potentially less precise adaptation.

This tutorial uses a LoRA rank of 4. In practice, begin with a relatively small rank (such as 4, 8, 16). This is computationally efficient for experimentation. Train your model with this rank and evaluate the performance improvement on your task. Gradually increase the rank in subsequent trials and see if that further boosts performance.

In [16]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

Note that enabling LoRA reduces the number of trainable parameters significantly (from 2.5 billion to 1.3 million).

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=30, batch_size=1)

Epoch 1/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 286s 732ms/step - loss: 0.4527 - sparse_categorical_accuracy: 0.7565
Epoch 2/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 270s 733ms/step - loss: 0.2729 - sparse_categorical_accuracy: 0.8242
Epoch 3/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 270s 733ms/step - loss: 0.2326 - sparse_categorical_accuracy: 0.8452
Epoch 4/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 270s 733ms/step - loss: 0.2178 - sparse_categorical_accuracy: 0.8534
Epoch 5/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 270s 733ms/step - loss: 0.2066 - sparse_categorical_accuracy: 0.8613
Epoch 6/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 270s 733ms/step - loss: 0.1973 - sparse_categorical_accuracy: 0.8674
Epoch 7/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 270s 733ms/step - loss: 0.1888 - sparse_categorical_accuracy: 0.8726
Epoch 8/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 323s 877ms/step - loss: 0.1813 - sparse_categorical_accuracy: 0.8773
Epoch 9/30
368/368 ━━━━━━━━━━━━━━━━━━━━ 270s 733ms/step - loss: 0.1731 - sparse_categorical_accuracy: 0.8816
Epoch 10/30
368/368

## Inference after fine-tuning
After fine-tuning, responses follow the instruction provided in the prompt.

### Europe Trip Prompt


In [31]:
# prompt = template.format(
#     instruction="What should I do on a trip to Europe?",
#     response="",
# )
# print(gemma_lm.generate(prompt, max_length=256))

The model now recommends places to visit in Europe.

In [21]:
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

### ELI5 Photosynthesis Prompt


In [48]:
prompt = template.format(
    instruction="you are an ITSM administrator expert in json response,you have to provide nine casues for following user query :Users are unable to access certain SAP modules or functions.response should be in json format:{'cause1':.. ,'cause2':.. , 'cause3':... ,...}",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
you are an ITSM administrator expert in json response,you have to provide nine casues for following user query :Users are unable to access certain SAP modules or functions.response should be in json format:{'cause1':.. ,'cause2':.. , 'cause3':... ,...}

Response:
{"cause1": "Incorrect user permissions", "cause2": "SAP modules or functions not properly configured", "cause3": "Network connectivity issues", "cause4": "Outdated SAP software", "cause5": "Insufficient memory allocated to SAP system", "cause6": "Incompatible browser or plugin", "cause7": "System downtime or maintenance", "cause8": "Security restrictions on SAP system", "cause9": "Third-party software conflicts with SAP system", "relevancy": "100%"},


In [34]:
gemma_lm.save('/kaggle/working/mygemma.h5')

ValueError: Unable to synchronously create dataset (name already exists)

The model now explains photosynthesis in simpler terms.

Note that for demonstration purposes, this tutorial fine-tunes the model on a small subset of the dataset for just one epoch and with a low LoRA rank value. To get better responses from the fine-tuned model, you can experiment with:

1. Increasing the size of the fine-tuning dataset
2. Training for more steps (epochs)
3. Setting a higher LoRA rank
4. Modifying the hyperparameter values such as `learning_rate` and `weight_decay`.


## Summary and next steps

This tutorial covered LoRA fine-tuning on a Gemma model using KerasNLP. Check out the following docs next:

* Learn how to [generate text with a Gemma model](https://ai.google.dev/gemma/docs/get_started).
* Learn how to perform [distributed fine-tuning and inference on a Gemma model](https://ai.google.dev/gemma/docs/distributed_tuning).
* Learn how to [use Gemma open models with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma).
* Learn how to [fine-tune Gemma using KerasNLP and deploy to Vertex AI](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb).
